In [55]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline


In [56]:
import pandas_profiling as pp

In [57]:
data_train=pd.read_csv('train.csv')

In [58]:
valid_part=0.3

In [59]:
data_train.head(10)

,Id,target,1,2,3,4,5,6,7
0,0,5.85,4.80,5.85,NaN,NaN,NaN,NaN,NaN
1,3,4.28,5.88,6.84,NaN,NaN,NaN,NaN,NaN
2,4,3.97,3.20,2.70,NaN,NaN,NaN,NaN,NaN
3,5,5.95,7.70,7.06,NaN,NaN,NaN,NaN,NaN
4,6,4.70,5.50,5.30,NaN,NaN,NaN,NaN,NaN
5,7,4.36,4.91,4.48,NaN,NaN,NaN,NaN,NaN
6,8,3.74,3.71,3.76,NaN,NaN,NaN,NaN,NaN
7,9,7.70,3.75,3.60,NaN,NaN,NaN,NaN,NaN
8,10,3.34,2.13,5.40,NaN,NaN,NaN,NaN,NaN
9,11,3.96,2.24,4.22,NaN,NaN,NaN,NaN,NaN


In [60]:
data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147 entries, 0 to 146
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Id      147 non-null    int64  
 1   target  147 non-null    float64
 2   1       145 non-null    float64
 3   2       145 non-null    float64
 4   3       32 non-null     float64
 5   4       31 non-null     float64
 6   5       33 non-null     float64
 7   6       37 non-null     float64
 8   7       37 non-null     float64
dtypes: float64(8), int64(1)
memory usage: 10.5 KB


In [61]:
pp.ProfileReport(data_train)

Summarize dataset:   0%|          | 0/21 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [62]:
dt = data_train.drop(['Id','3','4','5','6','7'], axis = 1)
dt = dt.dropna()
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144 entries, 0 to 146
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   target  144 non-null    float64
 1   1       144 non-null    float64
 2   2       144 non-null    float64
dtypes: float64(3)
memory usage: 4.5 KB


In [63]:
dt.head(5)

,target,1,2
0,5.85,4.80,5.85
1,4.28,5.88,6.84
2,3.97,3.20,2.70
3,5.95,7.70,7.06
4,4.70,5.50,5.30


# Preparing to modeling

In [64]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV,StratifiedKFold

In [65]:
targetn='target'

In [66]:
train0b = dt
train_target0b = train0b[targetn]
train0b = train0b.drop([targetn], axis=1)
# Synthesis valid as test for selection models
trainb, testb, targetb, target_testb = train_test_split(train0b, train_target0b, test_size=valid_part, random_state=0)

In [70]:
train_target0 = dt[targetn]
train0 = dt.drop([targetn], axis=1)

In [74]:
scaler = StandardScaler()
train0 = pd.DataFrame(scaler.fit_transform(train0), columns = train0.columns)

In [75]:
train0.head(3)

,1,2
0,-0.085038,0.464653
1,0.443331,0.963267
2,-0.867807,-1.121847


In [76]:
len(train0)

144

In [95]:
train,test,target,target_test=train_test_split(train0,train_target0,test_size=valid_part,random_state=0)

In [96]:
train.head(3)

,1,2
95,-0.804207,1.018669
92,1.284809,0.238010
50,1.064655,0.640931


In [97]:
test.head(3)

,1,2
7,-0.598730,-0.668561
89,-1.993038,-1.877323
97,-0.466638,-0.482211


In [98]:
train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 95 to 47
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       100 non-null    float64
 1   2       100 non-null    float64
dtypes: float64(2)
memory usage: 2.3 KB


In [99]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44 entries, 7 to 112
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   1       44 non-null     float64
 1   2       44 non-null     float64
dtypes: float64(2)
memory usage: 1.0 KB


In [104]:
# models
from sklearn.linear_model import LinearRegression, SGDRegressor, RidgeCV
from sklearn.svm import SVR, LinearSVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor 
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, VotingRegressor 
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
import sklearn.model_selection
from sklearn.model_selection import cross_val_predict as cvp
from sklearn import metrics
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import xgboost as xgb
import lightgbm as lgb

# model tuning
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe, space_eval

import warnings
warnings.filterwarnings("ignore")
